# llama_3.1 8B Instruct

This model could ben downloaded and deploy locally, despite its slow runtime. 

In [1]:
import json
import torch
import transformers
from huggingface_hub import snapshot_download, login

c:\Users\brian\Desktop\mlp-confidence-calibration-research\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
with open("api.json") as f:
    login(token=json.load(f)["HF_TOKEN"])

In [3]:
local_dir = snapshot_download(repo_id="meta-llama/Llama-3.2-3B-Instruct")
local_dir

Fetching 16 files: 100%|██████████| 16/16 [00:00<00:00, 76871.55it/s]


'C:\\Users\\brian\\.cache\\huggingface\\hub\\models--meta-llama--Llama-3.2-3B-Instruct\\snapshots\\0cb88a4f764b7a12671c53f0838cd831a0843b95'

In [4]:

pipeline: transformers.pipeline = transformers.pipeline("text-generation", model=local_dir, device_map="auto", model_kwargs={"torch_dtype": torch.float16, "low_cpu_mem_usage": True})
terminators = [pipeline.tokenizer.eos_token_id, pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")]

Loading checkpoint shards: 100%|██████████| 2/2 [00:10<00:00,  5.11s/it]
Some parameters are on the meta device because they were offloaded to the cpu.
Device set to use cuda:0


In [24]:
# takes a str query and returns str responses
def get_response(query, message_history=[], max_tokens=2048, temperature=0.6, top_p=0.9):
    user_prompt = message_history + [{"role": "user", "content": query}]
    prompt = pipeline.tokenizer.apply_chat_template(
        user_prompt, tokenize=False, add_generation_prompt=True
    )
    outputs = pipeline(
        prompt,
        max_new_tokens=max_tokens,
        eos_token_id=terminators,
        do_sample=True,
        temperature=temperature,
        top_p=top_p
    )
    response = outputs[0]["generated_text"][len(prompt):]
    return response

# take a list of messages (dict[str, any] -> role, content) and a response
def response_from_msg_list(msg_list: list[dict], message_history=[], max_tokens=2048, temperature=0.6, top_p=0.9):
    flat_msg_list = []
    for x in range(len(msg_list)): flat_msg_list += [{"role":k , "content": v} for k, v in msg_list[x].items()]
    user_prompt = message_history + flat_msg_list
    # print("message list:", user_prompt)
    prompt = pipeline.tokenizer.apply_chat_template(
        user_prompt, tokenize=False, add_generation_prompt=True
    )
    outputs = pipeline(
        prompt,
        max_new_tokens=max_tokens,
        eos_token_id=terminators,
        do_sample=True,
        temperature=temperature,
        top_p=top_p
    )
    response = outputs[0]["generated_text"][len(prompt):]
    return response

In [25]:
response_from_msg_list([{"user": "How many gold medals did Kristin Otto win at the 1987 European Championships?"}, {"system": "as concisely as possible and your confidence as a percentage in parentheses at the end"}])

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


"I couldn't find any information about a person named Kristin Otto winning gold medals at the 1987 European Championships. (0%)"